In [2]:
import torch as T
import torch.nn as nn 
import os
project_path= os.path.dirname(os.path.abspath(os.curdir))
import sys
sys.path.insert(0, project_path+ '/Tools')
from ActorCriticNetworks import ActorNetwork, CriticNetwork

In [3]:
test_state= T.rand((5, 4))
test_action= T.rand((5, 2))

In [4]:
test_critic1= CriticNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='test_critic1', learning_rate=0.02, project_path=project_path)
test_critic1.save_checkpoint()
print(test_critic1(test_state, test_action))
test_critic1.load_checkpoint()
print(test_critic1(test_state, test_action))

... saving checkpoint ...
tensor([[-0.0138],
        [-0.0755],
        [-0.0602],
        [-0.0609],
        [ 0.0111]], grad_fn=<AddmmBackward0>)
... loading checkpoint ...
tensor([[-0.0138],
        [-0.0755],
        [-0.0602],
        [-0.0609],
        [ 0.0111]], grad_fn=<AddmmBackward0>)


In [5]:
test_actor1= ActorNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), 'test_actor', 0.02, project_path=project_path)
test_actor1.save_checkpoint()
print(test_actor1(test_state))
test_actor1.load_checkpoint()
print(test_actor1(test_state))

... saving checkpoint ...
tensor([[ 0.5501,  0.1159],
        [ 0.4632,  0.5521],
        [ 0.5391,  0.3004],
        [ 0.5998, -0.0442],
        [ 0.5526,  0.1891]], grad_fn=<AddmmBackward0>)
... loading checkpoint ...
tensor([[ 0.5501,  0.1159],
        [ 0.4632,  0.5521],
        [ 0.5391,  0.3004],
        [ 0.5998, -0.0442],
        [ 0.5526,  0.1891]], grad_fn=<AddmmBackward0>)


**3. Functional tests** to check if this module functions as intended.They only verify the output of an action and do not check the intermediate states of the system when performing that action. 

Can copy parameters

In [6]:
test_critic2= CriticNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='test_critic2', learning_rate=0.02, project_path=project_path)

print(test_critic2(test_state, test_action))
for target_param, param in zip(test_critic2.parameters(), test_critic1.parameters()):
    target_param.data.copy_(param.data)
print(test_critic2(test_state, test_action))

tensor([[0.7321],
        [0.6102],
        [0.7516],
        [0.7558],
        [0.6097]], grad_fn=<AddmmBackward0>)
tensor([[-0.0138],
        [-0.0755],
        [-0.0602],
        [-0.0609],
        [ 0.0111]], grad_fn=<AddmmBackward0>)


In [7]:
test_actor2= ActorNetwork(4, 2, (10, 15), (nn.ReLU(), nn.ReLU()), name='test_actor2', learning_rate=0.02, project_path=project_path)
print(test_actor2(test_state))
for target_param, param in zip(test_actor2.parameters(), test_actor1.parameters()):
    target_param.data.copy_(param.data)
print(test_actor2(test_state))

tensor([[ 0.9180, -0.7236],
        [ 1.0393, -0.1689],
        [ 0.8875, -0.6562],
        [ 0.9131, -0.6161],
        [ 0.9137, -0.6883]], grad_fn=<AddmmBackward0>)
tensor([[ 0.5501,  0.1159],
        [ 0.4632,  0.5521],
        [ 0.5391,  0.3004],
        [ 0.5998, -0.0442],
        [ 0.5526,  0.1891]], grad_fn=<AddmmBackward0>)


Can learn to approximate values

In [8]:
y_train= T.rand((5, 1))
epochs = 300
for epoch in range(epochs):
    y_pred = test_critic1(test_state, test_action)
    loss = T.nn.MSELoss()(y_pred, y_train)
    test_critic1.optimizer.zero_grad()
    loss.backward()
    test_critic1.optimizer.step()

test_critic1(test_state, test_action), y_train

(tensor([[0.6293],
         [0.9279],
         [0.0703],
         [0.1535],
         [0.3170]], grad_fn=<AddmmBackward0>),
 tensor([[0.6293],
         [0.9279],
         [0.0703],
         [0.1535],
         [0.3170]]))

In [9]:
epochs = 300
for epoch in range(epochs):
    y_pred = test_actor1(test_state)
    loss = T.nn.MSELoss()(y_pred, test_action)
    test_actor1.optimizer.zero_grad()
    loss.backward()
    test_actor1.optimizer.step()

test_actor1(test_state), test_action

(tensor([[0.5472, 0.4789],
         [0.6220, 0.2908],
         [0.8849, 0.7271],
         [0.8436, 0.8526],
         [0.4033, 0.0236]], grad_fn=<AddmmBackward0>),
 tensor([[0.5473, 0.4789],
         [0.6221, 0.2908],
         [0.8849, 0.7271],
         [0.8436, 0.8526],
         [0.4034, 0.0237]]))